In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

In [ ]:
previsores = pd.read_csv('/content/drive/MyDrive/DeepLearning/Data/1.1 entradas_breast.csv.csv')

In [ ]:
classe = pd.read_csv('/content/drive/MyDrive/DeepLearning/Data/1.1 saidas_breast.csv.csv')

Vamos criar nossa primeira função de classificação  em sequencia

In [ ]:
def Rede1():
  clas = Sequential()
  clas.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
  clas.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
  clas.add(Dense(units=1, activation='sigmoid'))
  opt = keras.optimizers.Adam(lr=0.001, decay=0.0001, clipvalue=0.5)
  clas.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
  return clas

In [ ]:
clas1 = KerasClassifier(build_fn=Rede1, epochs=100, batch_size=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [ ]:
results1 = cross_val_score(estimator=clas1, X=previsores, y=classe, cv=10, scoring='accuracy')

In [ ]:
results1

array([0.66666667, 0.78947368, 0.87719298, 0.68421053, 0.84210526,
       0.85964912, 0.84210526, 0.89473684, 0.75438596, 0.82142857])

In [ ]:
media1 =  results1.mean()
media1

0.8031954887218046

In [ ]:
desvio1 = results1.std()
desvio1

0.07468001454343363

Vamos testar alterando coma função Dropout e zerando 20% dos nossos valores.

In [ ]:
def Rede2():
  clas = Sequential()
  clas.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
  clas.add(Dropout(0.2))
  clas.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
  clas.add(Dense(units=1, activation='sigmoid'))
  clas.add(Dropout(0.2))
  opt = keras.optimizers.Adam(lr=0.001, decay=0.0001, clipvalue=0.5)
  clas.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
  return clas

In [ ]:
clas2 = KerasClassifier(build_fn=Rede2, epochs=100, batch_size=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [ ]:
results2 = cross_val_score(estimator=clas2, X=previsores, y=classe, cv=10, scoring='accuracy')

In [ ]:
results2

array([0.19298246, 0.61403509, 0.63157895, 0.50877193, 0.50877193,
       0.78947368, 0.71929825, 0.77192982, 0.77192982, 0.76785714])

In [ ]:
media2 =  results2.mean()
media2

0.6276629072681705

In [ ]:
desvio2 = results2.std()
desvio2

0.1769967089453397

Vamos criar uma função para descobrir os melhores parametros com o GridSearchCV



In [ ]:
def Rede3(optimizer, loos, kernel_initializer, activation, neurons):
  clas = Sequential()
  clas.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer, input_dim=30))
  clas.add(Dropout(0.2))
  clas.add(Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer))
  clas.add(Dense(units=1, activation='sigmoid'))
  clas.add(Dropout(0.2))
  clas.compile(optimizer=optimizer, loss=loos, metrics=['binary_accuracy'])
  return clas

In [ ]:
clas3 = KerasClassifier(build_fn=Rede3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [ ]:
parametros = {'batch_size':[10, 30],
              'epochs':[50, 100],
              'optimizer':['adam', 'sgd'],
              'loos':['binary_crossentropy', 'hinge'],
              'kernel_initializer':['random_uniform', 'normal'],
              'activation':['relu', 'tahn'],
              'neurons':[16, 8]}

In [ ]:
gridsearch = GridSearchCV(estimator=clas3, param_grid=parametros, scoring='accuracy', cv=5)

In [ ]:
gridsearch = gridsearch.fit(previsores, classe)

In [ ]:
best = gridsearch.best_params_
best

{'activation': 'relu',
 'batch_size': 10,
 'epochs': 100,
 'kernel_initializer': 'random_uniform',
 'loos': 'binary_crossentropy',
 'neurons': 8,
 'optimizer': 'adam'}

In [ ]:
best_score = gridsearch.best_score_
best_score

0.906831237385499

Vamos utilizar os melhores parametros para fazer nossa classificação

In [ ]:
classificador = Sequential()
classificador.add(Dense(units=8, activation='relu', kernel_initializer='normal', input_dim=30))
classificador.add(Dropout(0.2))
classificador.add(Dense(units=8, activation='relu', kernel_initializer='normal'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units=1, activation='sigmoid'))
classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [ ]:
classificador.fit(previsores, classe, batch_size=10, epochs=100)

Epoch 1/100
57/57 [==============================] - 0s 1ms/step - loss: 0.3311 - binary_accuracy: 0.8682
Epoch 2/100
57/57 [==============================] - 0s 1ms/step - loss: 0.3473 - binary_accuracy: 0.8489
Epoch 3/100
57/57 [==============================] - 0s 1ms/step - loss: 0.3396 - binary_accuracy: 0.8682
Epoch 4/100
57/57 [==============================] - 0s 1ms/step - loss: 0.3366 - binary_accuracy: 0.8770
Epoch 5/100
57/57 [==============================] - 0s 1ms/step - loss: 0.3223 - binary_accuracy: 0.8717
Epoch 6/100
57/57 [==============================] - 0s 1ms/step - loss: 0.3562 - binary_accuracy: 0.8524
Epoch 7/100
57/57 [==============================] - 0s 2ms/step - loss: 0.3320 - binary_accuracy: 0.8735
Epoch 8/100
57/57 [==============================] - 0s 2ms/step - loss: 0.3344 - binary_accuracy: 0.8770
Epoch 9/100
57/57 [==============================] - 0s 1ms/step - loss: 0.3150 - binary_accuracy: 0.8787
Epoch 10/100
57/57 [==========================

Vamos criar uma base para testar nossa rede neural

In [ ]:
new = np.array([[15.80, 8.35, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.138, 0.20, 0.05, 1098, 0.87, 4500, 
                 0.15, 0.560, 1300, 0.25, 0.101, 563, 1.25, 35, 298, 0.147, 15.10, 23.15, 0.69, 112, 10, 88]])

In [ ]:
previsao = classificador.predict(new)
previsao

array([[1.]], dtype=float32)

Vamos salvar nossa rede neural e os seus respctivos pesos

In [ ]:
classificador_json = classificador.to_json()
classificador_json

'{"class_name": "Sequential", "config": {"name": "sequential_665", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 30], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_1355_input"}}, {"class_name": "Dense", "config": {"name": "dense_1355", "trainable": true, "batch_input_shape": [null, 30], "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "RandomNormal", "config": {"mean": 0.0, "stddev": 0.05, "seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dropout", "config": {"name": "dropout_670", "trainable": true, "dtype": "float32", "rate": 0.2, "noise_shape": null, "seed": null}}, {"class_name": "Dense", "config": {"name": "dense_1356", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu",

In [ ]:
with open('classificador_json.json', 'w') as json_file:
  json_file.write(classificador_json)

In [ ]:
classificador.save_weights('classificador_json.h5')